# Pré-Processamento de Dados em Python com Scikit-Learn

Neste notebook, vamos abordar as principais etapas de pré-processamento, incluindo:

- **Tratamento de valores ausentes**
- **Detecção e remoção de outliers**
- **Normalização e padronização**
- **Codificação de dados categóricos**
- **Tratamento de inconsistências**
- **Transformação de características**

## 1. Importando bibliotecas

Vamos iniciar importando as bibliotecas principais que utilizaremos:

In [9]:
%pip install numpy pandas matplotlib seaborn scikit-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, PolynomialFeatures
from scipy import stats

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

print("Bibliotecas importadas com sucesso!")

  Using cached numpy-2.2.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pandas-2.2.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached matplotlib-3.10.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.6.1-cp313-cp313-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached contourpy-1.3.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.6-cp313-cp313-macosx_10_13_universal2.whl.metadata (166 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pillow-11.1.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using 

## 2. Criando um dataset de exemplo

Para ilustrar as etapas de pré-processamento, vamos criar um dataset fictício com:
- Valores numéricos, alguns ausentes (NaN)
- Dados categóricos
- Valores outliers
- Coluna com tipo de dado inconsistente
- Possibilidade de criar novas features

In [10]:
# Para consistência de exemplo, definimos uma semente aleatória
np.random.seed(42)

# Número de registros
n = 20

# Criando dados numéricos
idade = np.random.randint(18, 70, size=n).astype(float)
# Inserindo alguns valores ausentes aleatórios
idade[np.random.choice(n, 3, replace=False)] = np.nan

# Criando dados categóricos
genero = np.random.choice(['Masculino', 'Feminino', 'NaoBinario'], size=n)

# Criando outra feature com dados numéricos, mas com um outlier
salario = np.random.randint(2000, 10000, size=n).astype(float)
# Inserindo um outlier artificial
salario[0] = 100000.0  # Grande outlier

# Criando dados inconsistentes: coluna com números como string
altura_str = [str(val) for val in np.random.randint(150, 200, size=n)]
# Simulando que alguns valores aparecem com vírgula e não ponto
altura_str[5] = altura_str[5].replace('1', '1,')
altura_str[10] = altura_str[10] + ' cm'  # outra inconsistência

# Montando o DataFrame
df = pd.DataFrame({
    'Idade': idade,
    'Genero': genero,
    'Salario': salario,
    'Altura': altura_str
})

df

,Idade,Genero,Salario,Altura
0,56.0,NaoBinario,100000.0,163
1,69.0,Masculino,3585.0,180
2,46.0,Masculino,5943.0,197
3,32.0,NaoBinario,9555.0,164
4,NaN,Feminino,5073.0,157
5,25.0,Masculino,3021.0,"1,63"
6,38.0,Feminino,5461.0,172
7,56.0,Feminino,4613.0,189
8,36.0,Feminino,7225.0,170
9,40.0,Masculino,5843.0,165


Vamos verificar como o nosso dataset ficou. Observe que temos:
- Alguns valores ausentes em `Idade` (NaN);
- Um outlier no `Salario` (100000.0);
- Coluna `Altura` como string, com formatos diferentes (ex.: 'cm', vírgula etc.).

## 3. Tratamento de valores ausentes

Técnicas comuns:
- **Remoção** de linhas ou colunas
- **Imputação** por média ou mediana
- **Imputação** por métodos mais avançados (modelos preditivos, KNN, etc.)

Aqui, usaremos o `SimpleImputer` do scikit-learn para substituição por média ou mediana. Também é possível usar `fillna` do pandas em muitos casos.

In [11]:
# Verificando quantidade de valores ausentes
df.isnull().sum()

Idade      3
Genero     0
Salario    0
Altura     0
dtype: int64

### 3.1 Imputação por média

In [12]:
# Vamos criar uma cópia do dataset para aplicar a imputação.
df_imputed_mean = df.copy()

# SimpleImputer com estratégia de imputar a média.
imputer_mean = SimpleImputer(strategy='mean')

# O SimpleImputer só funciona com dados numéricos, então selecionamos apenas a coluna "Idade".
df_imputed_mean['Idade'] = imputer_mean.fit_transform(df_imputed_mean[['Idade']])

df_imputed_mean

,Idade,Genero,Salario,Altura
0,56.000000,NaoBinario,100000.0,163
1,69.000000,Masculino,3585.0,180
2,46.000000,Masculino,5943.0,197
3,32.000000,NaoBinario,9555.0,164
4,39.588235,Feminino,5073.0,157
5,25.000000,Masculino,3021.0,"1,63"
6,38.000000,Feminino,5461.0,172
7,56.000000,Feminino,4613.0,189
8,36.000000,Feminino,7225.0,170
9,40.000000,Masculino,5843.0,165


### 3.2 Imputação por mediana
Vamos tentar outra estratégia de imputação. Substituir por mediana pode ser mais robusto quando temos outliers em uma coluna.

In [13]:
# Criando outra cópia
df_imputed_median = df.copy()
imputer_median = SimpleImputer(strategy='median')
df_imputed_median['Idade'] = imputer_median.fit_transform(df_imputed_median[['Idade']])

df_imputed_median

,Idade,Genero,Salario,Altura
0,56.0,NaoBinario,100000.0,163
1,69.0,Masculino,3585.0,180
2,46.0,Masculino,5943.0,197
3,32.0,NaoBinario,9555.0,164
4,40.0,Feminino,5073.0,157
5,25.0,Masculino,3021.0,"1,63"
6,38.0,Feminino,5461.0,172
7,56.0,Feminino,4613.0,189
8,36.0,Feminino,7225.0,170
9,40.0,Masculino,5843.0,165


A escolha da estratégia (média, mediana ou outra) depende do contexto do seu dataset. Em algumas situações, pode ser melhor remover linhas, em outras pode-se aplicar modelos de imputação mais avançados.

## 4. Detecção e remoção de outliers

Valores atípicos (outliers) podem enviesar sua análise e prejudicar o aprendizado do modelo. A detecção pode ser feita de várias formas, por exemplo:
- Distância interquartil (IQR)
- Z-score
- Modelos de detecção de anomalias (Isolation Forest, etc.)

Aqui, vamos usar o método IQR e remover os valores que estão além de 1.5 * IQR (faixa interquartil).

In [14]:
df_outliers_removed = df.copy()

# Primeiro, precisamos lidar com valores ausentes antes da análise de outliers.
# Para simplificar, vamos imputar a Idade pela mediana aqui.
df_outliers_removed['Idade'] = imputer_median.fit_transform(df_outliers_removed[['Idade']])

# Vamos detectar outliers na coluna Salario usando IQR
Q1 = df_outliers_removed['Salario'].quantile(0.25)
Q3 = df_outliers_removed['Salario'].quantile(0.75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

print("Faixa Interquartil (IQR):", IQR)
print("Limite inferior:", limite_inferior)
print("Limite superior:", limite_superior)

# Filtrando os outliers
condicao = (df_outliers_removed['Salario'] >= limite_inferior) & (df_outliers_removed['Salario'] <= limite_superior)
df_outliers_removed = df_outliers_removed[condicao]

df_outliers_removed

Faixa Interquartil (IQR): 4010.5
Limite inferior: -2133.75
Limite superior: 13908.25


,Idade,Genero,Salario,Altura
1,69.0,Masculino,3585.0,180
2,46.0,Masculino,5943.0,197
3,32.0,NaoBinario,9555.0,164
4,40.0,Feminino,5073.0,157
5,25.0,Masculino,3021.0,"1,63"
6,38.0,Feminino,5461.0,172
7,56.0,Feminino,4613.0,189
8,36.0,Feminino,7225.0,170
9,40.0,Masculino,5843.0,165
10,28.0,Feminino,9989.0,194 cm


Podemos observar que o salário 100000 (outlier óbvio) é removido. Em cenários reais, pode ser mais apropriado transformá-lo, corrigi-lo ou investigar a origem do valor antes de simplesmente removê-lo.

## 5. Normalização e Padronização

Tanto a **normalização** (ex: MinMaxScaler) quanto a **padronização** (ex: StandardScaler) são usadas para ajustar a escala dos dados. Muitos algoritmos de machine learning são sensíveis a escalas diferentes.

- **Normalização (Min-Max)**: transforma os valores para um intervalo [0, 1].
- **Padronização (StandardScaler)**: subtrai a média e divide pelo desvio padrão, resultando em uma distribuição com média 0 e desvio padrão 1.

In [15]:
# Vamos criar duas colunas para comparar Normalização e Padronização.
df_scaled = df_outliers_removed[['Idade', 'Salario']].copy()

# Imputando idade antes de escalar (caso ainda tenha NaNs, mas removemos outliers apenas do Salario)
df_scaled['Idade'] = imputer_median.fit_transform(df_scaled[['Idade']])

# Normalização Min-Max
minmax_scaler = MinMaxScaler()
df_scaled_minmax = minmax_scaler.fit_transform(df_scaled)

# Padronização
standard_scaler = StandardScaler()
df_scaled_standard = standard_scaler.fit_transform(df_scaled)

# Transformando em DataFrames para visualização
df_scaled_minmax = pd.DataFrame(df_scaled_minmax, columns=['Idade_normalizada', 'Salario_normalizado'])
df_scaled_standard = pd.DataFrame(df_scaled_standard, columns=['Idade_padronizada', 'Salario_padronizado'])

# Concatenando para comparar lado a lado
df_compare_scaling = pd.concat([df_scaled.reset_index(drop=True), df_scaled_minmax, df_scaled_standard], axis=1)
df_compare_scaling

,Idade,Salario,Idade_normalizada,Salario_normalizado,Idade_padronizada,Salario_padronizado
0,69.0,3585.0,1.00,0.181911,2.473127,-0.987976
1,46.0,5943.0,0.54,0.483137,0.590276,0.040972
2,32.0,9555.0,0.26,0.944558,-0.555807,1.617122
3,40.0,5073.0,0.42,0.371998,0.099097,-0.338665
4,25.0,3021.0,0.12,0.109862,-1.128849,-1.234086
5,38.0,5461.0,0.38,0.421564,-0.064629,-0.169355
6,56.0,4613.0,0.74,0.313235,1.408907,-0.539393
7,36.0,7225.0,0.34,0.646909,-0.228355,0.600392
8,40.0,5843.0,0.42,0.470363,0.099097,-0.002664
9,28.0,9989.0,0.18,1.000000,-0.883260,1.806504


Observe como os valores são transformados em escalas diferentes dependendo da técnica usada. A escolha de qual scaler usar depende muito do algoritmo de machine learning e do contexto do problema.

## 6. Codificação de dados categóricos

Algoritmos de ML precisam de valores numéricos. Para isso, podemos:
- **Label Encoding**: cada categoria recebe um inteiro.
- **One-Hot Encoding**: cada categoria gera uma nova coluna binária (0 ou 1).

Exemplo: se temos a coluna `Genero` com valores `['Masculino', 'Feminino', 'NaoBinario']`, podemos transformá-la em colunas dummy ou inteiros.

In [18]:
# Label Encoding
df_cat = df_outliers_removed.copy()
# Antes de tudo, vamos garantir que Idade não tenha valores ausentes.
df_cat['Idade'] = imputer_median.fit_transform(df_cat[['Idade']])

le = LabelEncoder()
df_cat['Genero_label'] = le.fit_transform(df_cat['Genero'])

df_cat

,Idade,Genero,Salario,Altura,Genero_label
1,69.0,Masculino,3585.0,180,1
2,46.0,Masculino,5943.0,197,1
3,32.0,NaoBinario,9555.0,164,2
4,40.0,Feminino,5073.0,157,0
5,25.0,Masculino,3021.0,"1,63",1
6,38.0,Feminino,5461.0,172,0
7,56.0,Feminino,4613.0,189,0
8,36.0,Feminino,7225.0,170,0
9,40.0,Masculino,5843.0,165,1
10,28.0,Feminino,9989.0,194 cm,0


In [19]:
# One-Hot Encoding com pandas
df_cat_onehot = pd.get_dummies(df_cat, columns=['Genero'], prefix='Genero')

df_cat_onehot

,Idade,Salario,Altura,Genero_label,Genero_Feminino,Genero_Masculino,Genero_NaoBinario
1,69.0,3585.0,180,1,False,True,False
2,46.0,5943.0,197,1,False,True,False
3,32.0,9555.0,164,2,False,False,True
4,40.0,5073.0,157,0,True,False,False
5,25.0,3021.0,"1,63",1,False,True,False
6,38.0,5461.0,172,0,True,False,False
7,56.0,4613.0,189,0,True,False,False
8,36.0,7225.0,170,0,True,False,False
9,40.0,5843.0,165,1,False,True,False
10,28.0,9989.0,194 cm,0,True,False,False


Observe que agora temos colunas como `Genero_Feminino`, `Genero_Masculino` e `Genero_NaoBinario` valendo 0 ou 1.
Já com **LabelEncoder**, a coluna `Genero_label` possui valores inteiros que representam cada categoria.

A escolha de uma ou outra abordagem depende do tipo de algoritmo e se há ordem lógica entre as categorias.

## 7. Tratamento de inconsistências

Dados podem vir de diferentes fontes, com formatos incorretos, tipagem errada, etc. Neste exemplo, a coluna `Altura` está como string, com diferentes padrões. Vamos corrigir para tipo numérico (float).

In [20]:
# Exemplo de tratamento para a coluna "Altura"
df_incons = df.copy()

# Observando os valores de Altura
print("Valores originais:")
print(df_incons['Altura'].values)

# 1) Remover a string " cm" se existir
df_incons['Altura'] = df_incons['Altura'].str.replace(' cm', '', regex=False)

# 2) Trocar vírgula por ponto
df_incons['Altura'] = df_incons['Altura'].str.replace(',', '.', regex=False)

# 3) Converter para float
df_incons['Altura'] = pd.to_numeric(df_incons['Altura'], errors='coerce')

# Verificando resultado
print("\nValores após tratamento:")
print(df_incons['Altura'].values)

# Exibindo dataframe final
df_incons

Valores originais:
['163' '180' '197' '164' '157' '1,63' '172' '189' '170' '165' '194 cm'
 '167' '196' '173' '175' '174' '194' '190' '178' '164']

Valores após tratamento:
[163.   180.   197.   164.   157.     1.63 172.   189.   170.   165.
 194.   167.   196.   173.   175.   174.   194.   190.   178.   164.  ]


,Idade,Genero,Salario,Altura
0,56.0,NaoBinario,100000.0,163.00
1,69.0,Masculino,3585.0,180.00
2,46.0,Masculino,5943.0,197.00
3,32.0,NaoBinario,9555.0,164.00
4,NaN,Feminino,5073.0,157.00
5,25.0,Masculino,3021.0,1.63
6,38.0,Feminino,5461.0,172.00
7,56.0,Feminino,4613.0,189.00
8,36.0,Feminino,7225.0,170.00
9,40.0,Masculino,5843.0,165.00


Agora, temos a coluna `Altura` em formato numérico (float). Com isso, podemos usá-la em qualquer cálculo ou análise.

Em casos de formatação incorreta, podemos utilizar expressões regulares ou funções de parsing mais complexas, dependendo da fonte dos dados.

## 8. Transformação de características

A ideia aqui é criar novas features a partir das existentes. Exemplos:
- Criar variáveis polinomiais (PolynomialFeatures)
- Aplicar transformações logarítmicas, raiz quadrada etc.
- Combinar colunas existentes para criar variáveis compostas.

Vamos exemplificar a criação de variáveis polinomiais para `Idade` e `Altura` usando *PolynomialFeatures* do scikit-learn.

In [21]:
# Para isso, vamos reusar o df_incons onde a Altura já foi corrigida.
df_transformed = df_incons.copy()

# Precisamos tratar valores ausentes em "Idade" novamente.
df_transformed['Idade'] = imputer_median.fit_transform(df_transformed[['Idade']])

# Selecionando as colunas numéricas
features = df_transformed[['Idade', 'Altura']].dropna()

poly = PolynomialFeatures(degree=2, include_bias=False)
features_poly = poly.fit_transform(features)

df_features_poly = pd.DataFrame(
    features_poly,
    columns=["Idade", "Altura", "Idade^2", "Idade*Altura", "Altura^2"][:features_poly.shape[1]]
)

df_features_poly

,Idade,Altura,Idade^2,Idade*Altura,Altura^2
0,56.0,163.00,3136.0,9128.00,26569.0000
1,69.0,180.00,4761.0,12420.00,32400.0000
2,46.0,197.00,2116.0,9062.00,38809.0000
3,32.0,164.00,1024.0,5248.00,26896.0000
4,40.0,157.00,1600.0,6280.00,24649.0000
5,25.0,1.63,625.0,40.75,2.6569
6,38.0,172.00,1444.0,6536.00,29584.0000
7,56.0,189.00,3136.0,10584.00,35721.0000
8,36.0,170.00,1296.0,6120.00,28900.0000
9,40.0,165.00,1600.0,6600.00,27225.0000


Observe que ganhamos colunas como `Idade^2`, `Idade*Altura` e `Altura^2`. Em muitos modelos (como regressão linear), essas transformações podem melhorar a performance se houver relação não-linear entre as variáveis.

Também podemos usar transformações logarítmicas para lidar com distribuições muito dispersas, ou qualquer outra transformação que faça sentido no contexto do problema.

Ou podemos fazer apenas utilizando Pandas.

In [23]:
features_mod = features.copy()
features_mod['Altura_x_Idade'] = features_mod['Idade'] * features_mod['Altura']

features_mod

,Idade,Altura,Altura_x_Idade
0,56.0,163.00,9128.00
1,69.0,180.00,12420.00
2,46.0,197.00,9062.00
3,32.0,164.00,5248.00
4,40.0,157.00,6280.00
5,25.0,1.63,40.75
6,38.0,172.00,6536.00
7,56.0,189.00,10584.00
8,36.0,170.00,6120.00
9,40.0,165.00,6600.00


# Conclusão

Neste notebook, vimos diversas etapas de pré-processamento de dados:

1. **Tratamento de valores ausentes**: imputação por média, mediana ou outros métodos.
2. **Detecção e remoção de outliers**: uso do IQR para ilustrar a remoção de valores extremos.
3. **Normalização e padronização**: uso de `MinMaxScaler` e `StandardScaler` para ajustar escalas.
4. **Codificação de dados categóricos**: `LabelEncoder` e `OneHotEncoder` (ou `pd.get_dummies`).
5. **Tratamento de inconsistências**: correção de formatos incorretos e conversão de tipo.
6. **Transformação de características**: criação de novas features a partir das existentes.

Todas essas etapas ajudam a garantir uma melhor qualidade dos dados antes de alimentar os algoritmos de machine learning.